In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops 

from collections import Counter 
import pandas as pd
import numpy as np
import time
import os

import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [3]:
# dir path for sample dataset
dir_path = os.getcwd() 
dataset_path = dir_path + '\\Dataset\\'
dataset2_path = dir_path + '\\Dataset2\\'

In [4]:
X = []
Y = []

x_true = []
x_false = []

In [5]:
for folderName, subfolders, filenames in os.walk(dataset_path):
    for subfolder in subfolders:
        for _,sF,_ in os.walk(dataset_path+subfolder):
            if(len(sF) != 0):
                for _,_,im in os.walk(dataset_path+subfolder+'\\'+sF[0]+'\\'):
                    for i in im:
                        temp_img = image.load_img(dataset_path + subfolder + '\\' + sF[0] + '\\' + i,target_size=(224,224))
                        temp_img = image.img_to_array(temp_img)
                        X.append(temp_img)
                        x_true.append(temp_img)
                        Y.append(1)
                for _,_,im in os.walk(dataset_path+subfolder+'\\'+sF[1]+'\\'):
                    for i in im:
                        temp_img = image.load_img(dataset_path + subfolder + '\\' + sF[1] + '\\' + i,target_size=(224,224))
                        temp_img = image.img_to_array(temp_img)
                        X.append(temp_img)
                        x_false.append(temp_img)
                        Y.append(0)

In [6]:
x_true = np.array(x_true)
x_false = np.array(x_false)

In [7]:
x_true.shape

(360, 224, 224, 3)

In [8]:
X = np.array(X)
Y = np.array(Y)

In [9]:
X.shape

(720, 224, 224, 3)

In [10]:
X /= 255

In [11]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=True, # set input mean to 0 over the dataset
        samplewise_center=False, # set each sample mean to 0
        featurewise_std_normalization=False, # divide inputs by std of the dataset
        samplewise_std_normalization=False, # divide each input by its std
        zca_whitening=False, # apply ZCA whitening
        rotation_range=15, # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.05, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.05, # randomly shift images vertically (fraction of total height)
        horizontal_flip=False, # randomly flip images
        vertical_flip=False,   # randomly flip images
        fill_mode='nearest') 
datagen.fit(X)

In [12]:
from sklearn.model_selection import train_test_split
train_x,test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.06, random_state = 1)

In [13]:
train_x.shape

(676, 224, 224, 3)

In [14]:
test_x.shape

(44, 224, 224, 3)

In [15]:
from keras.utils.np_utils import to_categorical
Train_Y = to_categorical(train_y,num_classes=2)
# Val_Y = to_categorical(val_y,num_classes=2)
Test_Y = to_categorical(test_y,num_classes=2)

<b><i>Class 1 - Forged</i></b> <br>
<b><i>Class 2 - Genuine</i></b>

In [16]:
# vgg16 model 
model_vgg = VGG16(weights='imagenet', include_top=False)

In [17]:
# predicting model 
model=Sequential()

model.add(Dense(350,input_dim=3584,activation='relu',kernel_initializer='uniform'))
keras.layers.core.Dropout(0.3, noise_shape=None, seed=None)

model.add(Dense(50,input_dim=350,activation='relu',kernel_initializer='uniform'))
keras.layers.core.Dropout(0.4, noise_shape=None, seed=None)

model.add(Dense(10,input_dim=50,activation='relu',kernel_initializer='uniform'))
keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)

model.add(Dense(units=2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer="sgd",metrics=['accuracy'])

In [18]:
counter_idx = 0

In [19]:
32*100

3200

In [ ]:
for batch_x, batch_y in datagen.flow(train_x,Train_Y,batch_size=32):
    
    if batch_x.shape[0] == 4:
        continue
#     print(batch_x.shape)
#     print(batch_y.shape)
    
    features_batch = model_vgg.predict(batch_x)
    Train_X = features_train.reshape(224,3584)
    loss = model.train(Train_X,batch_y,epochs=10,batch_size=8)
    
    if counter_idx >= 100:
        break
    
    counter_idx += 1

In [ ]:
for batch_x,batch_y in datagen.flow(train_x,train_y,batch_size=32):
    features_train = 